In [1]:
from __future__ import print_function, division, absolute_import
import argparse
import os
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import sys

sys.path.append('.')
import pretrainedmodels
import pretrainedmodels.utils

In [2]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = args.lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [ ]:
def validate(val_loader, a1, model_1, a2, model_2, criterion):
    with torch.no_grad():
        batch_time = AverageMeter()
        losses = AverageMeter()
        top1 = AverageMeter()
        top5 = AverageMeter()

        # switch to evaluate mode
        model_1.eval()
        model_2.eval()

        end = time.time()
        for i, (input, target) in enumerate(val_loader):
            target = target.cuda()
            input = input.cuda()

            # compute output
            output_1 = model_1(input)
            output_2 = model_2(input)
            loss_1 = criterion(output_1, target)
            loss_2 = criterion(output_2, target)
            
            # fuse
            output = a1 * output_1 + a2 * output_2
            
            # measure accuracy and record loss
            prec1, prec5 = accuracy(output.data, target.data, topk=(1, 5))
            loss = a1 * loss_1 + a2 * loss_2 
            losses.update(loss.data.item(), input.size(0))
            top1.update(prec1.item(), input.size(0))
            top5.update(prec5.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % 10 == 0:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                      'Acc@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                       i, len(val_loader), batch_time=batch_time, loss=losses,
                       top1=top1, top5=top5))

        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

        return top1.avg, top5.avg

In [ ]:
print("=> using pre-trained parameters '{}'".format('imagenet'))
a1 = 0.5002
model_1 = pretrainedmodels.__dict__['resnet18'](num_classes=1000,pretrained='imagenet')
a2 = 0.4998
model_2 = pretrainedmodels.__dict__['vgg11'](num_classes=1000,pretrained='imagenet')

In [3]:
cudnn.benchmark = True
data_path ='/media/k1407/DA18EBFA09C1B27D/ImageNet/imagenet2012'
valdir = os.path.join(data_path, 'val')

In [ ]:
scale = 0.875

print('Images transformed from size {} to {}'.format(
    int(round(max(model_1.input_size) / scale)),
    model_1.input_size))

val_tf = pretrainedmodels.utils.TransformImage(
    model_1,
    scale=scale,
    preserve_aspect_ratio=True
)

val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(valdir, val_tf),
    batch_size=20, shuffle=False,
    num_workers=4, pin_memory=True)

In [ ]:
criterion = nn.CrossEntropyLoss().cuda()
model_1 = torch.nn.DataParallel(model_1).cuda()
model_2 = torch.nn.DataParallel(model_2).cuda()
validate(val_loader, a1, model_1, a2, model_2, criterion)

In [ ]:
a1 = 0.5042
a2 = 0.4958
validate(val_loader, a1, model_1, a2, model_2, criterion)

In [ ]:
validate(val_loader, a1, model_1, a2, model_2, criterion)

In [4]:
def validate_2(val_loader_1, val_loader_2, a1, model_1, a2, model_2, criterion):
    with torch.no_grad():
        batch_time = AverageMeter()
        losses = AverageMeter()
        top1 = AverageMeter()
        top5 = AverageMeter()

        # switch to evaluate mode
        model_1.eval()
        model_2.eval()

        end = time.time()
        for i, data in enumerate(zip(val_loader_1, val_loader_2)):
            inputs1 = data[0][0].cuda();
            labels1 = data[0][1].cuda();
            inputs2 = data[1][0].cuda();
            labels2 = data[1][1].cuda();
            
            # compute output
            output_1 = model_1(inputs1)
            output_2 = model_2(inputs2)
            loss_1 = criterion(output_1, labels1)
            loss_2 = criterion(output_2, labels2)
            
            # fuse
            output = a1 * output_1 + a2 * output_2
            
            # measure accuracy and record loss
            prec1, prec5 = accuracy(output.data, labels1.data, topk=(1, 5))
            loss = a1 * loss_1 + a2 * loss_2 
            losses.update(loss.data.item(), inputs1.size(0))
            top1.update(prec1.item(), inputs1.size(0))
            top5.update(prec5.item(), inputs1.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % 10 == 0:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                      'Acc@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                       i, len(val_loader_1), batch_time=batch_time, loss=losses,
                       top1=top1, top5=top5))

        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

        return top1.avg, top5.avg

In [7]:
scale = 0.875
print("=> using pre-trained parameters '{}'".format('imagenet'))
a1 = 0.4657
model_1 = pretrainedmodels.__dict__['vgg19'](num_classes=1000,pretrained='imagenet')
a2 = 0.5343
model_2 = pretrainedmodels.__dict__['vgg19_bn'](num_classes=1000,pretrained='imagenet')

print('Images transformed from size {} to {}'.format(
    int(round(max(model_1.input_size) / scale)),
    model_1.input_size))

val_tf_1 = pretrainedmodels.utils.TransformImage(
    model_1,
    scale=scale,
    preserve_aspect_ratio=True
)

val_tf_2 = pretrainedmodels.utils.TransformImage(
    model_2,
    scale=scale,
    preserve_aspect_ratio=True
)

val_loader_1 = torch.utils.data.DataLoader(
    datasets.ImageFolder(valdir, val_tf_1),
    batch_size=20, shuffle=False,
    num_workers=4, pin_memory=True)

val_loader_2 = torch.utils.data.DataLoader(
    datasets.ImageFolder(valdir, val_tf_2),
    batch_size=20, shuffle=False,
    num_workers=4, pin_memory=True)
criterion = nn.CrossEntropyLoss().cuda()
model_1 = torch.nn.DataParallel(model_1).cuda()
model_2 = torch.nn.DataParallel(model_2).cuda()
validate_2(val_loader_1,val_loader_2, a1, model_1, a2, model_2, criterion)

=> using pre-trained parameters 'imagenet'
Images transformed from size 256 to [3, 224, 224]
Test: [0/2500]	Time 1.774 (1.774)	Loss 0.9383 (0.9383)	Acc@1 80.000 (80.000)	Acc@5 95.000 (95.000)
Test: [10/2500]	Time 0.143 (0.401)	Loss 0.5216 (0.6229)	Acc@1 95.000 (85.000)	Acc@5 100.000 (96.818)
Test: [20/2500]	Time 0.970 (0.383)	Loss 0.9386 (0.7610)	Acc@1 90.000 (82.619)	Acc@5 95.000 (95.714)
Test: [30/2500]	Time 0.144 (0.343)	Loss 0.3470 (0.6000)	Acc@1 90.000 (85.484)	Acc@5 100.000 (96.935)
Test: [40/2500]	Time 0.922 (0.338)	Loss 0.6895 (0.5053)	Acc@1 85.000 (87.683)	Acc@5 90.000 (97.317)
Test: [50/2500]	Time 0.142 (0.321)	Loss 0.4646 (0.4816)	Acc@1 90.000 (88.333)	Acc@5 95.000 (97.255)
Test: [60/2500]	Time 0.941 (0.323)	Loss 0.4621 (0.4844)	Acc@1 95.000 (88.689)	Acc@5 95.000 (97.377)
Test: [70/2500]	Time 0.145 (0.315)	Loss 0.2571 (0.5090)	Acc@1 95.000 (88.028)	Acc@5 100.000 (97.324)
Test: [80/2500]	Time 0.730 (0.313)	Loss 2.2027 (0.5449)	Acc@1 45.000 (87.346)	Acc@5 90.000 (97.037)
Test:

Test: [800/2500]	Time 0.146 (0.289)	Loss 0.2328 (0.7808)	Acc@1 95.000 (81.061)	Acc@5 100.000 (95.811)
Test: [810/2500]	Time 0.691 (0.288)	Loss 0.4184 (0.7751)	Acc@1 90.000 (81.196)	Acc@5 100.000 (95.845)
Test: [820/2500]	Time 0.142 (0.289)	Loss 0.5322 (0.7713)	Acc@1 90.000 (81.340)	Acc@5 90.000 (95.859)
Test: [830/2500]	Time 0.404 (0.289)	Loss 0.1995 (0.7716)	Acc@1 95.000 (81.312)	Acc@5 100.000 (95.854)
Test: [840/2500]	Time 0.143 (0.288)	Loss 0.6210 (0.7671)	Acc@1 90.000 (81.439)	Acc@5 90.000 (95.868)
Test: [850/2500]	Time 0.453 (0.288)	Loss 0.0721 (0.7653)	Acc@1 100.000 (81.528)	Acc@5 100.000 (95.881)
Test: [860/2500]	Time 0.145 (0.287)	Loss 0.4952 (0.7657)	Acc@1 95.000 (81.539)	Acc@5 100.000 (95.889)
Test: [870/2500]	Time 0.271 (0.287)	Loss 0.7739 (0.7648)	Acc@1 45.000 (81.521)	Acc@5 100.000 (95.901)
Test: [880/2500]	Time 0.143 (0.286)	Loss 0.4767 (0.7611)	Acc@1 95.000 (81.561)	Acc@5 100.000 (95.948)
Test: [890/2500]	Time 0.686 (0.287)	Loss 1.5587 (0.7611)	Acc@1 55.000 (81.538)	Acc@

Test: [1610/2500]	Time 0.144 (0.281)	Loss 0.9247 (0.9737)	Acc@1 85.000 (77.297)	Acc@5 90.000 (93.575)
Test: [1620/2500]	Time 0.144 (0.281)	Loss 1.8636 (0.9744)	Acc@1 55.000 (77.310)	Acc@5 80.000 (93.553)
Test: [1630/2500]	Time 0.151 (0.280)	Loss 1.8643 (0.9797)	Acc@1 65.000 (77.207)	Acc@5 95.000 (93.495)
Test: [1640/2500]	Time 0.145 (0.280)	Loss 2.2488 (0.9838)	Acc@1 50.000 (77.142)	Acc@5 90.000 (93.464)
Test: [1650/2500]	Time 0.145 (0.280)	Loss 1.5935 (0.9870)	Acc@1 70.000 (77.062)	Acc@5 80.000 (93.428)
Test: [1660/2500]	Time 0.144 (0.280)	Loss 2.0610 (0.9894)	Acc@1 45.000 (76.987)	Acc@5 85.000 (93.420)
Test: [1670/2500]	Time 0.147 (0.280)	Loss 0.1695 (0.9921)	Acc@1 95.000 (76.927)	Acc@5 100.000 (93.408)
Test: [1680/2500]	Time 0.144 (0.280)	Loss 0.4612 (0.9889)	Acc@1 90.000 (77.002)	Acc@5 95.000 (93.430)
Test: [1690/2500]	Time 0.578 (0.280)	Loss 2.6558 (0.9929)	Acc@1 55.000 (76.904)	Acc@5 80.000 (93.400)
Test: [1700/2500]	Time 0.144 (0.280)	Loss 0.9887 (0.9967)	Acc@1 75.000 (76.869)	A

Test: [2420/2500]	Time 0.144 (0.280)	Loss 2.5110 (1.0788)	Acc@1 35.000 (75.242)	Acc@5 80.000 (92.414)
Test: [2430/2500]	Time 0.145 (0.280)	Loss 1.2495 (1.0815)	Acc@1 55.000 (75.169)	Acc@5 100.000 (92.392)
Test: [2440/2500]	Time 0.145 (0.280)	Loss 1.1586 (1.0818)	Acc@1 60.000 (75.158)	Acc@5 95.000 (92.399)
Test: [2450/2500]	Time 0.143 (0.279)	Loss 0.4271 (1.0836)	Acc@1 90.000 (75.100)	Acc@5 95.000 (92.391)
Test: [2460/2500]	Time 0.142 (0.279)	Loss 0.1269 (1.0833)	Acc@1 100.000 (75.116)	Acc@5 100.000 (92.389)
Test: [2470/2500]	Time 0.145 (0.279)	Loss 0.3178 (1.0807)	Acc@1 90.000 (75.152)	Acc@5 95.000 (92.406)
Test: [2480/2500]	Time 0.143 (0.279)	Loss 1.1036 (1.0775)	Acc@1 90.000 (75.238)	Acc@5 90.000 (92.432)
Test: [2490/2500]	Time 0.145 (0.280)	Loss 0.8320 (1.0749)	Acc@1 85.000 (75.303)	Acc@5 95.000 (92.453)
 * Acc@1 75.256 Acc@5 92.440


(75.256, 92.44)